In [3]:
import datetime
start_time = datetime.datetime.now(); print(start_time)

2018-09-19 14:41:39.311068


In [4]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }<\style>"))

In [17]:
from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd ; pd.set_option("display.max_columns",200)
import pickle
import statsmodels.api as sm

In [9]:
data_prepared_for_analysis_store_path = u'C:/Users/IMPTEMP_A_PACIFIC/Desktop/EDP_2016/Data/hdf/result.h5'

In [6]:
result_store_path_csv = 'C:/Users/IMPTEMP_A_PACIFIC/Desktop/EDP_2016/Data/result/csv/'

In [7]:
df_2011 = pd.read_csv(result_store_path_csv+ "step_3_{}.csv".format(2011))

C:\Users\IMPTEMP_A_PACIFIC\Desktop\WinPython-64bit-3.6.3.0Qt5\python-3.6.3.amd64\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (141) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [10]:
df_2014 = pd.read_csv(result_store_path_csv+ "step_3_{}.csv".format(2014))

C:\Users\IMPTEMP_A_PACIFIC\Desktop\WinPython-64bit-3.6.3.0Qt5\python-3.6.3.amd64\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (141,160) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [11]:
df_2014.shape

(2478701, 162)

In [42]:
df = pd.merge(df_2011,df_2014, on = 'ID_FISC_LOG_DIFF', suffixes=("_2011", ("_2014")))

In [43]:
df.shape

(1741300, 323)

In [48]:
df_save = df.copy()

In [73]:
df = df_save.copy()

In [74]:
df["Rbg_2011"] =(( df.ZTSAM_2011*0.9) + df.ZRAGM_2011 + df.ZRACM_2011 + df.ZRNCM_2011 + df.ZFONM_2011 + df.ZPERM_2011*0.9
                    + df.ZRICM_2011)
df["Rbg_2014"] =(( df.ZTSAM_2014*0.9) + df.ZRAGM_2014 + df.ZRACM_2014 + df.ZRNCM_2014 + df.ZFONM_2014 + df.ZPERM_2014*0.9
                    + df.ZRICM_2014)


df["Delta_rbg"] = df.Rbg_2014 - df.Rbg_2011
df["Rbg_splines"]  = pd.cut(df.Rbg_2011, 10)

In [75]:
for nb_enf in range(1,7): ### in modification
    df["Between_threshold_{}_child".format(nb_enf)
          ] = ((df.Rbg_2011>  threshold_byr_rbg["2014"].iloc[nb_enf-1])
               & (df.Rbg_2011< threshold_byr_rbg["2011"].iloc[nb_enf-1]))

In [76]:
df.Between_threshold_1_child.value_counts()

False    1689838
True       51462
Name: Between_threshold_1_child, dtype: int64

In [77]:
for nb_enf in range(1,7): ### in modification
    df["Over_threshold_{}_child".format(nb_enf)
          ] =  (df.Rbg_2011 > threshold_byr_rbg["2011"].iloc[nb_enf-1])

In [78]:
threshold_byr_rbg = pd.DataFrame()
threshold_byr_rbg["2011"] = pd.Series([63233,73516,94451,115185,135941,150684])
threshold_byr_rbg["2014"] = pd.Series([58291,63530,73806,84103,94368,104633])

In [79]:
nb_enf_max_for_annalysis = 6 
between_threshold_for_child_rank = ["Between_threshold_{}_child".format(nb_child) for nb_child in range(1,nb_enf_max_for_annalysis+1)]
over_threshold_for_child_rank = ["Over_threshold_{}_child".format(nb_child) for nb_child in range(1,nb_enf_max_for_annalysis+1)]
children_number =  ["child_{}".format(nb_child) for nb_child in range(1,nb_enf_max_for_annalysis+1)]

In [80]:
interaction_child_between_threshold  = ["{}:{}".format(child,interaction) for child, interaction in zip(children_number, between_threshold_for_child_rank)]
interaction_child_over_threshold  = ["{}:{}".format(child,interaction) for child, interaction in zip(children_number, over_threshold_for_child_rank)]

In [81]:
for col in df.columns: print(col) 

Unnamed: 0_2011
level_0_2011
index_2011
ID_FISC_LOG_DIFF
AN_FISC_x_2011
AN_REVENU_2011
REG_2011
DEP_x_2011
DEPCOM_2011
TU10_2011
NBFOYM_2011
NBPERSM_2011
INF14M_2011
SUP14M_2011
NBERAM_2011
NB_UC_2011
AGERF_2011
SEXERF_2011
OCCTYP_2011
TYPMEN9_2011
I_CHAMPM_2011
I_PAUVRE50M_2011
I_PAUVRE60M_2011
NB_ALLOC_CNAF_2011
NB_ALLOC_CNAV_2011
NB_ALLOC_MSAF_2011
NB_ALLOC_MSAV_2011
NIVVIEM_2011
CENTILE_2011
REVDISPM_2011
REVPERM_2011
REVINIM_2011
REVDECM_2011
ZTSAM_2011
ZSALM_2011
ZCHOM_2011
ZPERM_2011
ZRETM_2011
ZRSTM_2011
ZALRM_2011
ZRTOM_2011
ZRAGM_2011
ZRICM_2011
ZRNCM_2011
ZFONM_2011
ZVAMM0_2011
ZVAMM_2011
ZVALM0_2011
ZVALM_2011
ZRACM_2011
ZETRM_2011
ZALVM_2011
PSOCM_2011
PFAMM_2011
M_PNAM_2011
M_PAJEM_2011
M_CLCAM_2011
M_COLCAM_2011
M_AFM_2011
M_CFM_2011
M_ARSM_2011
M_AEEHM_2011
M_ASFM_2011
MINIM_2011
M_MINVM_2011
M_RSAM_2011
M_RSA_ACTM_2011
M_RSA_SOCM_2011
M_RSA_PFAM_2011
M_AAH_CAAHM_2011
LOGTM_2011
M_ALFM_2011
M_APLM_2011
M_ALSM_2011
M_ALSETM_2011
M_PPEM_2011
ZIMPOM_2011
ZIMPVALM_2011
ZTHA

In [82]:
year = 2011
to_load_id_log_2011 = ["couple_child_detached_or_attached_id_fisc_log_{}.p".format(year),
                  "couple_mar_pac_child_maj_ratach_id_fisc_log_{}.p".format(year),
                  "couple_mar_pac_child_ss_maj_id_fisc_log_{}.p".format(year),
                  "couple_mar_pac_ss_enf_id_fisc_log_{}.p".format(year),
                  "celib_ss_enf_id_fisc_log_{}.p".format(year),
                  "celib_ac_enf_ss_maj_id_fisc_log_{}.p".format(year),
                  "celib_with_detached_child_id_fisc_log_{}.p".format(year),
                  "single_ac_single_ac_enf_ac_maj_ratach_id_fisc_log_{}.p".format(year),
                  "concubin_with_or_without_children_id_fisc_log_{}.p".format(year),]
year = 2014
to_load_id_log_2014 = ["couple_child_detached_or_attached_id_fisc_log_{}.p".format(year),
                  "couple_mar_pac_child_maj_ratach_id_fisc_log_{}.p".format(year),
                  "couple_mar_pac_child_ss_maj_id_fisc_log_{}.p".format(year),
                  "couple_mar_pac_ss_enf_id_fisc_log_{}.p".format(year),
                  "celib_ss_enf_id_fisc_log_{}.p".format(year),
                  "celib_ac_enf_ss_maj_id_fisc_log_{}.p".format(year),
                  "celib_with_detached_child_id_fisc_log_{}.p".format(year),
                  "single_ac_single_ac_enf_ac_maj_ratach_id_fisc_log_{}.p".format(year),
                  "concubin_with_or_without_children_id_fisc_log_{}.p".format(year),]

In [83]:
#Charge à la fois dans l'environnement de travail et dans un dictionnaire...
pickle_dict = dict()
for pickle_to_load in to_load_id_log:
    loaded_pickle =  pickle.load(open(pickles_path+'{}'.format(pickle_to_load),'rb') )
    globals()[pickle_to_load[:-2]] = loaded_pickle
    print(pickle_to_load, "len:" , len(globals()[pickle_to_load[:-2]]));
    pickle_dict[pickle_to_load[:-2]] = loaded_pickle

NameError: name 'to_load_id_log' is not defined

In [84]:
df= df[df.TYPMEN9_2011 == df.TYPMEN9_2014]

In [85]:
df = df[df.TYPMEN9_2011.isin([30,41,42,43])]

In [86]:
df["Nb_child"] =  df.NBPAC_2011

In [96]:
df = df[df.NBFOYM_2011 == 1 ]
df = df[df.NBFOYM_2014 == 1 ]

In [97]:
for nbpac in range(1,7):
    df["child_{}".format(nbpac)] = df.NBPAC_2011

In [98]:
df = df.query("50000<Rbg_2011<200000")

In [99]:
m = sm.OLS.from_formula("(Delta_rbg)~   {}  ".format(
        ' + '.join(children_number+ #below_threshold_for_child_rank+interaction_child_below_thresholds
                  between_threshold_for_child_rank+children_number+ over_threshold_for_child_rank+
                  interaction_child_between_threshold+interaction_child_over_threshold))
                                                                                        , df.query("NBPAC_2011>=0").sample(frac = 1))
fit = m.fit()
summary = fit.summary(); print(summary)

                            OLS Regression Results                            
Dep. Variable:              Delta_rbg   R-squared:                       0.023
Model:                            OLS   Adj. R-squared:                  0.023
Method:                 Least Squares   F-statistic:                     166.5
Date:                Wed, 19 Sep 2018   Prob (F-statistic):               0.00
Time:                        16:23:13   Log-Likelihood:            -2.0185e+06
No. Observations:              176189   AIC:                         4.037e+06
Df Residuals:                  176163   BIC:                         4.037e+06
Df Model:                          25                                         
Covariance Type:            nonrobust                                         
                                                coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------

In [100]:
df["Delta_ZIMPOM"] = df.ZIMPOM_2014 - df.ZIMPFORM_2011

In [101]:
m = sm.OLS.from_formula("(Delta_rbg)~   {}  ".format(
        ' + '.join(children_number+ #below_threshold_for_child_rank+interaction_child_below_thresholds
                  between_threshold_for_child_rank+children_number+ over_threshold_for_child_rank+
                  interaction_child_between_threshold+interaction_child_over_threshold))
                                                                                        , df.query("NBPAC_2011>=0").sample(frac = 1))
fit = m.fit()
summary = fit.summary(); print(summary)

                            OLS Regression Results                            
Dep. Variable:              Delta_rbg   R-squared:                       0.023
Model:                            OLS   Adj. R-squared:                  0.023
Method:                 Least Squares   F-statistic:                     166.5
Date:                Wed, 19 Sep 2018   Prob (F-statistic):               0.00
Time:                        16:24:19   Log-Likelihood:            -2.0185e+06
No. Observations:              176189   AIC:                         4.037e+06
Df Residuals:                  176163   BIC:                         4.037e+06
Df Model:                          25                                         
Covariance Type:            nonrobust                                         
                                                coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------

In [ ]:
df[["Between_threshold_2_child", "Rbg_2011", "Delta_rbg"]]